# **Training and Evaluation with build-in methods**

## **Setup && Introduction**

In [1]:
import tensorboard

import tensorflow as tf
import tensorflow.keras as tfk
from tensorflow.keras import layers as lyrs

import numpy as np
import matplotlib.pyplot as plt

In [21]:
# HYPERPARAMETERS uniform through out the code
BS = 64 # Batch_Size
EP = 10 # Epochs
VB = 1 # Verbose

## **API overview: An End-to-End Example**

In [62]:
# Creating a basic Mnist Classifier for starting
(tr_data, tr_lbls), (ts_data, ts_lbls) = tfk.datasets.mnist.load_data()

tr_data = tr_data.reshape(-1, 28*28).astype('float32') / 255.0
ts_data = ts_data.reshape(-1, 28*28).astype('float32') / 255.0

val_data = tr_data[:-10000] # Reserve last 10,000 images for validation set
val_lbls = tr_lbls[:-10000] # Take last 10,000 labels as-well.

In [63]:
# Define the Model Structure and Compile
Model_mnist = tfk.Sequential(
    [
        lyrs.Dense(128, activation='relu', input_shape=(784,), name= 'InputLayer'),
        lyrs.Dense(64, activation='relu', name= 'Dense_1'),
        lyrs.Dense(10, activation='softmax', name= 'OutputLayer'),
    ], name= 'Digits_Recognition_Model'
)

Model_mnist.summary()

Model_mnist.compile(
    optimizer=tfk.optimizers.Nadam(),
    loss= tfk.losses.SparseCategoricalCrossentropy(),
    metrics= [tfk.metrics.SparseCategoricalAccuracy()]
)

Model: "Digits_Recognition_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputLayer (Dense)          (None, 128)               100480    
                                                                 
 Dense_1 (Dense)             (None, 64)                8256      
                                                                 
 OutputLayer (Dense)         (None, 10)                650       
                                                                 
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
_________________________________________________________________


In [64]:
# Training the Model and seeing the result!!
Model_mnist.fit(
    tr_data, tr_lbls,
    validation_data= (val_data, val_lbls),
    epochs= EP, verbose= VB,
    batch_size= BS
)

Epoch 1/10
240/938 [======>.......................] - ETA: 3s - loss: 0.5107 - sparse_categorical_accuracy: 0.8595

KeyboardInterrupt: 

In [25]:
# Evaluating the Model on Test Data!!
Model_mnist.evaluate(ts_data, ts_lbls, verbose= VB, batch_size= BS)

157/157 [==============================] - 1s 3ms/step - loss: 0.0784 - sparse_categorical_accuracy: 0.9785


[0.0783563107252121, 0.9785000085830688]

## **The Compile Method**

In [65]:
# A vauge Compile Function
Model_mnist.compile(
    optimizer= 'Adam',
    loss= 'MSE',
    metrics= ['accuracy']
)
# Typically, the compile function takes 3 main arguments
# OPTIMIZER, LOSS_FUNCTION,  and EVALUATION_METRIC(optional)

def get_model():
    Inp = tfk.Input(shape= (784, ))
    lyr = lyrs.Dense(64)(Inp)
    lyr = lyrs.Dense(64)(lyr)
    Out = lyrs.Dense(10)(lyr)

    return tfk.Model(inputs= Inp, outputs= Out)

def compile_model(model):
    model.compile(
        optimizer= 'Adam',
        loss= 'MSE',
        metrics= ['accuracy']
    )
    return model

# Once the Model is defined, it can be compiled
compiled_model = compile_model(get_model())

### **Built-in optimizers, losses and metrics**

**Visit `https://keras.io/api/` for in-depth information on inbuilt losses, metrics etc**

### **Custom Losses**
> **2 Methods of creating custom losses**
1. **Functional approach -- pass y_true and y_pred to a func and compute the difference between the two values**
2. **Sub-classing approach -- Implement methods __init__() and call()**

In [ ]:
# Functional Approach
def Mean_SQ_Error(y_true, y_pred):
    Squared_diff = tf.square(y_true, y_pred)
    loss = tf.reduce_mean(Squared_diff)
    return loss 

# Getting a Vauge Model calling a previously created function
model = get_model()

# Defining prams for compile
opt = tfk.optimizers.Adamax(0.001)
loss = Mean_SQ_Error
metric = tfk.metrics.SparseCategoricalAccuracy()

model.compile(
    optimizer= opt,
    loss= loss,
    metrics= [metric]
)

OneHot_lbls = tf.one_hot(tr_lbls, depth= 10)

model.fit(
    tr_data, tr_lbls, epochs= EP, verbose= 2, batch_size= BS
)
...

# Custom Loss not working so-far

In [ ]:
# Sub-Classing using tf.keras.losses.Loss class
class MeSqEr_Custom(tfk.losses.Loss):
    def init(self, regfac= 0.1):
        super().__init__()
        self.regularization_factor = regfac
    
    def call(self, ytrue, ypred):
        SqErr1 = tf.square(ytrue - ypred)
        mse = tf.reduce_mean(SqErr1)
        
        SqErr2 = tf.square(0.5 - ypred)
        reg = tf.reduce_mean(SqErr2)
        
        cstm_loss = mse + reg * self.regularization_factor
        return cstm_loss
    

class CustomMSE(tfk.losses.Loss):
    def __init__(self, regularization_factor=0.1, name="custom_mse"):
        super().__init__(name=name)
        self.regularization_factor = regularization_factor

    def call(self, y_true, y_pred):
        mse = tf.math.reduce_mean(tf.square(y_true - y_pred))
        reg = tf.math.reduce_mean(tf.square(0.5 - y_pred))
        return mse + reg * self.regularization_factor

# Get a Model    
model = get_model()

# Define prams
# optimizer and metric already defined from the last block: opt, metric
# cstm_mse = MeSqEr_Custom(0.25)
cstm_mse = CustomMSE()

model.compile(
    optimizer= opt,
    loss= cstm_mse,
    metrics= [metric]
)

tr_lbls_OneHot = tf.one_hot(tr_lbls, depth= 10)
model.fit(tr_data, tr_lbls_OneHot, epochs=EP, verbose=VB, batch_size=BS)

# Custom Loss not working so-far

### **Custom Metrics**

### **Handling losses and metrics that donot fit the standard signature**

### **Automatically setting apart a validation holdout set**

## **Training and Evaluation from tf.Datasets**


### **Using a Validation dataset**

## **Other supported input formats**

## **Using tfk.utils.Sequence object as input**

## **Using sample weighting and class weighting**

### **Class Weights**

### **Sample Weights**

## **Passing data to multi-input/output models**

## **Using Callbacks**

### **Built-in callbacks**


### **Writing custom callbacks**


## **Model Checkpoints**

## **Usign learning-rate schedules**

### **Passing a schedule to an optimizer**

### **Using callbacks to implement a dynamic learning rate schedule**

## **Visualizing loss and metrics during training**

### **Using Tensorboard callback**

# **Wrapup**